In [ ]:
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
import ISLP
from ISLP import load_data
from statsmodels.datasets import get_rdataset
from sklearn.cluster import KMeans

# Función de limpieza general: Creamos una unica base completa en un excel, con mismas columnas y en el mismo orden.
def limpieza_general():
    ind2004 = pd.read_stata('individual_t104.dta')
    ind2024 = pd.read_excel('usu_individual_T124.xlsx')
    ind2004.set_index('CODUSU', inplace=True)
    ind2024.set_index('CODUSU', inplace=True)
    ind2024 = ind2024.loc[:, ~ind2024.columns.str.contains("CH05")]
    ind2004.columns = ind2004.columns.str.upper()

    # Normaliza valores de REGION en ind2004 si son textos
    if 'REGION' in ind2004.columns:
        region_map = {
            'Gran Buenos Aires': 1,
            'NOA': 40,
            'NEA': 41,
            'Cuyo': 42,
            'Pampeana': 43,
            'Patagónica': 44
        }
        ind2004['REGION'] = ind2004['REGION'].replace(region_map)

    # Verifica que REGION esté en ind2024 y convertir a numérico
    if 'REGION' in ind2024.columns:
        ind2024['REGION'] = pd.to_numeric(ind2024['REGION'], errors='coerce')

    cols_2004 = set(ind2004.columns)
    cols_2024 = set(ind2024.columns)

    sobran_2004 = cols_2004 - cols_2024
    sobran_2024 = cols_2024 - cols_2004
    ind2004 = ind2004.drop(columns=sobran_2004)
    ind2024 = ind2024.drop(columns=sobran_2024)

    if ind2024.shape[1] == ind2004.shape[1]:
        print(
            f"Igual cantidad de columnas: {ind2004.shape[1]}")
    else:
        print(
            f'No son iguales: 2024: {ind2024.shape[1]}, 2004:{ind2004.shape[1]}')

    ind2024 = ind2024[ind2004.columns]

    base_completa = pd.concat([ind2004, ind2024], axis=0)
    variables_elegidas = ['ANO4', 'ESTADO', 'CH04', 'CH06', 'CH10', 'CH12', 'CH13', 'CH14', 'PP04A', 'PP04B_COD',
                          'PP07C', 'CAT_INAC', 'P21', 'PP3E_TOT', 'PP3F_TOT']
    base_completa = base_completa[variables_elegidas]
    base_completa.to_excel('BaseLimpiaEPH.xlsx', index=False)

    print("Archivo exportado a Excel.")


limpieza_general()

# Limpiamos la variable "CH06" (Edad) para que solo tenga valores numéricos.
def limpieza_edad():
    ind2004 = pd.read_excel('BaseLimpiaEPH.xlsx')
    valores_unicos_2004 = ind2004['CH06'][~ind2004['CH06'].apply(
        lambda x: isinstance(x, (int, float)))].unique()
    print("Valores no numéricos únicos en CH06 (2004):")
    print(valores_unicos_2004)

    año2004 = ind2004['ANO4'] == 2004
    no_numeros = ind2004.loc[año2004, 'CH06'][~ind2004.loc[año2004, 'CH06']
                                              .apply(lambda x: isinstance(x, (int, float)))].unique()
    print("Valores no numéricos únicos en CH06 (2004):", no_numeros)

    reemplazos = {'Menos de 1 año': 1, '98 y más años': 99}
    series_2004 = ind2004.loc[año2004, 'CH06'].astype(str).replace(reemplazos)
    ind2004.loc[año2004, 'CH06'] = pd.to_numeric(series_2004, errors='coerce')

    print('Valores de edad corregidos solo para 2004.')
    ind2004.to_excel('BaseLimpiaEPH.xlsx', index=False)
    print('Archivo guardado')


limpieza_edad()

# Limpiamos la variable "Estado" para que solo tenga valores numéricos en cuanto a la ocupación, desocupación y otros resultados.
def limpieza_estado():
    Est = pd.read_excel('BaseLimpiaEPH.xlsx')

    valores_unicos_2004 = Est['ESTADO'][~Est['ESTADO'].apply(
        lambda x: isinstance(x, (int, float)))].unique()
    print(
        f"Valores no numéricos únicos en ESTADO (2004):{valores_unicos_2004}")

    Est['ESTADO'] = Est['ESTADO'].astype(str)
    Est['ESTADO'] = Est['ESTADO'].replace({
        'Ocupado': 1,
        'Desocupado': 2,
        'Inactivo': 3,
        'Menor de 10 años': 4,
        'Entrevista individual no realizada (no respuesta al cuestion': 0})
    Est['ESTADO'] = pd.to_numeric(Est['ESTADO'], errors='coerce')

    print(f'Valores en ESTADO 2004:{Est['ESTADO'].unique()}')
    print(f'Tipo de valores: {Est['ESTADO'].dtype}')

    Est.to_excel('BaseLimpiaEPH.xlsx', index=False)
    print('Limpieza guardada.')


limpieza_estado()

# Luego de la limpieza, realizamos lo pedido:
# Creamos la variable "edad2", que es todas las edades elevadas al cuadrado, y añadimos la columna a la base.
def edad2():
    var_nueva = pd.read_excel('BaseLimpiaEPH.xlsx')
    var_nueva['edad2'] = np.square(var_nueva['CH06'])
    var_nueva.to_excel('BaseLimpiaEPH.xlsx')
    print('Nueva variable edad2 añadida.')


edad2()

# Creamos una función que, sobre la base general, crea un histograma de edades y un kernels de ocupados/desocupados.
def histo_kernel():
    df = pd.read_excel('BaseLimpiaEPH.xlsx')

    # Variables para el Histograma.
    edades2004 = df.loc[df['ANO4'] == 2004, 'CH06']
    edades2024 = df.loc[df['ANO4'] == 2024, 'CH06']
    df2004 = pd.DataFrame({'CH06': edades2004, 'ANO4': '2004'})
    df2024 = pd.DataFrame({'CH06': edades2024, 'ANO4': '2024'})
    combinado_edades = pd.concat([df2004, df2024], ignore_index=True)
    promedio_2004 = edades2004.mean()
    promedio_2024 = edades2024.mean()

    # Figura.
    fig, axes = plt.subplots(1, 2, figsize=(10, 6))

    # Histograma.
    sns.histplot(data=combinado_edades, x='CH06', ax=axes[0], hue='ANO4', bins=25, multiple='stack', palette={
        '2004': '#5799c6', '2024': 'orange'})
    axes[0].set_title('Distribución de edad EPH 2004/2024')
    axes[0].set_xlabel('Edad')
    axes[0].set_ylabel('Frecuencia')
    axes[0].axvline(promedio_2004, linestyle='dashed',
                    linewidth=1, color='red')
    axes[0].axvline(promedio_2024, linestyle='dashed',
                    linewidth=1, color='green')
    axes[0].set_xticks(np.arange(0, 101, 10))
    # Se usa patches y line2D para personalizar los labels.
    parche_2004 = mpatches.Patch(color='#5799c6', label='2004')
    parche_2024 = mpatches.Patch(color='orange', label='2024')
    line_prom04 = Line2D([0], [0], color='red',
                         linestyle='--', label='Promedio 2004')
    line_prom24 = Line2D([0], [0], color='green',
                         linestyle='--', label='Promedio 2024')
    axes[0].legend(handles=[parche_2004, parche_2024, line_prom04,
                            line_prom24], title='Año y Promedios')

    # Kernels.
    ocu2004 = df.loc[(df['ESTADO'] == 1) & (
        # Filtramos el Estado para ocupados/desocupados, y 2004/2024.
        df['ANO4'] == 2004), 'CH06']
    desoc2004 = df.loc[(df['ESTADO'] == 2) & (
        df['ANO4'] == 2004), 'CH06']
    ocu2024 = df.loc[(df['ESTADO'] == 1) & (
        df['ANO4'] == 2024), 'CH06']
    desoc2024 = df.loc[(df['ESTADO'] == 2) & (
        df['ANO4'] == 2024), 'CH06']
    sns.kdeplot(ocu2004, ax=axes[1], label='Ocupados 2004',
                fill=True, color='red', bw_method='scott')
    sns.kdeplot(desoc2004, ax=axes[1], label='Desocupados 2004',
                fill=True, color='grey', bw_method='scott')
    sns.kdeplot(ocu2024, ax=axes[1], label='Ocupados 2024',
                fill=True, color='green', bw_method='scott', linestyle='dashed')
    sns.kdeplot(desoc2024, ax=axes[1], label='Desocupados 2024',
                fill=True, color='grey', bw_method='scott', linestyle='dashed')
    axes[1].set_title('Estimación de Edad por Estado Laboral 2004/2024')
    axes[1].set_xlabel('Edad')
    axes[1].set_ylabel('Densidad estimada')
    axes[1].legend(title='Grupo')

    plt.tight_layout()
    plt.show()
    plt.savefig('Histo_Kernels.png')


histo_kernel()

# Limpiamos base en las variables "CH10", "CH12", "CH13" y "CH14" para crear la variable "educ" (Cant. de años de educación.)
def educ_limpieza():
    df = pd.read_excel('BaseLimpiaEPH.xlsx')
    df['CH10'] = df['CH10'].astype(str)
    df['CH10'] = df['CH10'].replace({
        'No asiste, pero asistió': 2,
        'Sí, asiste': 1,
        'Nunca asistió': 3,
        'Ns./Nr.': 9})
    df['CH10'] = pd.to_numeric(df['CH10'], errors='coerce')

    df['CH12'] = df['CH12'].astype(str).dropna()
    df['CH12'] = df['CH12'].replace({
        'Secundario': 4,
        'Primario': 2,
        'Universitario': 7,
        'Terciario': 6,
        'Jardín/Preescolar': 1,
        'Polimodal': 5,
        'EGB': 3,
        'Educación especial (discapacitado)': 9,
        'Ns./Nr.': 9,
        '99': 9})
    df['CH12'] = pd.to_numeric(df['CH12'], errors='coerce')

    df['CH13'] = df['CH13'].astype(str)
    df['CH13'] = df['CH13'].replace({
        'No': 2,
        'Sí': 1,
        'Ns./Nr.': 9})
    df['CH13'] = pd.to_numeric(df['CH13'], errors='coerce')

    df['CH14'] = df['CH14'].replace({i: f'0{i}' for i in range(0, 10)})
    df['CH14'] = pd.to_numeric(df['CH14'], errors='coerce')

    # Chequeamos los valores unicos de cada variable luego de limpiar.
    print(f'Valores en CH10:{df['CH10'].unique()}')
    print(f'Valores en CH12:{df['CH12'].unique()}')
    print(f'Valores en CH13:{df['CH13'].unique()}')
    print(f'Valores en CH14:{df['CH14'].unique()}')
    df.to_excel('BaseLimpiaEPH.xlsx', index=False)

    # Creamos una función que, en caso de que asista o hayaa asistido a alguna institución de educación, -
    # nos devuelva distintos valores en cada una. Si asiste o asistió a secundaria como última institución -
    # se hace 6 (Por primaria) + el último año cursado.

educ_limpieza()

def educ():
    df = pd.read_excel('BaseLimpiaEPH.xlsx')

    def educ(row):
        if row['CH10'] in [1]:  # Si asiste a alguna institución:
            if row['CH14'] in [99, 98]:
                # Educación especial.
                return 6
            else:
                # No contamos jardín como años de educación.
                if row['CH12'] == 1:
                    return 0
                # Primaria, al consultar por ultimo año cursado, esa es su cantidad de años.
                elif row['CH12'] == 2:
                    return row['CH14']
                # EGB, cant. de años maximo es 9, dirá los años cursados en su respuesta de 'CH14'
                elif row['CH12'] == 3:
                    return row['CH14']
                # Secundaria, 6 de primaria + ultimo año cursado.
                elif row['CH12'] == 4:
                    return 6 + row['CH14']
                # Polimodal, 8 años de EGB + ultimo año cursado.
                elif row['CH12'] == 5:
                    return 9 + row['CH14']
                # Terciario, 12 años de primaria y secundaria + ultimo año cursado.
                elif row['CH12'] == 6:
                    return 12 + row['CH14']
                elif row['CH12'] == 7:  # Univ, lo mismo que terciario.
                    return 12 + row['CH14']
                # Posgrado, asumo 5 años de educación universitaria: 17 + ultimo año cursado
                elif row['CH12'] == 8:
                    return 17 + row['CH14']
                elif row['CH12'] == 9:  # Ed. especial, asumo 6.
                    return row['CH14']
                else:
                    return 0
        elif row['CH10'] in [2]:
            if row['CH13'] in [1]:
                if row['CH14'] in [99, 98]:
                    # Educación especial.
                    return 6
                else:
                    # No contamos jardín como años de educación.
                    if row['CH12'] == 1:
                        return 0
                    # Primaria
                    elif row['CH12'] == 2:
                        return 6
                    # EGB, cant. de años maximo es 9
                    elif row['CH12'] == 3:
                        return 9
                    # Secundaria, 6 de primaria + 6
                    elif row['CH12'] == 4:
                        return 12
                    # Polimodal, 8 años de EGB + 3
                    elif row['CH12'] == 5:
                        return 12
                    # Terciario, 12 años de primaria y secundaria + ultimo año cursado.
                    elif row['CH12'] == 6:
                        return 15
                    elif row['CH12'] == 7:  # Univ, lo mismo que terciario.
                        return 17
                    # Posgrado, asumo 5 años de educación universitaria: 17 + 1 de posgrado.
                    elif row['CH12'] == 8:
                        return 18
                    elif row['CH12'] == 9:  # Ed. especial.
                        return 6
                    else:
                        return 0
            elif row['CH13'] in [2]:
                if row['CH14'] in [99, 98]:
                    # Educación especial.
                    return 6
                else:
                    # No contamos jardín como años de educación.
                    if row['CH12'] == 1:
                        return 0
                    # Primaria, al consultar por ultimo año cursado, esa es su cantidad de años.
                    elif row['CH12'] == 2:
                        return row['CH14']
                    # EGB, cant. de años maximo es 9, dirá los años cursados en su respuesta de 'CH14'
                    elif row['CH12'] == 3:
                        return row['CH14']
                    # Secundaria, 6 de primaria + ultimo año cursado.
                    elif row['CH12'] == 4:
                        return 6 + row['CH14']
                    # Polimodal, 9 años de EGB + ultimo año cursado.
                    elif row['CH12'] == 5:
                        return 9 + row['CH14']
                    # Terciario, 12 años de primaria y secundaria + ultimo año cursado.
                    elif row['CH12'] == 6:
                        return 12 + row['CH14']
                    elif row['CH12'] == 7:  # Univ, lo mismo que terciario.
                        return 12 + row['CH14']
                    # Posgrado, asumo 5 años de educación universitaria: 17 + ultimo año cursado
                    elif row['CH12'] == 8:
                        return 17 + row['CH14']
                    elif row['CH12'] == 9:  # Ed. especial.
                        return row['CH14']
                    else:
                        return 0
        elif row['CH10'] in [3]:  # Si nunca asistió, 0 años de educación.
            return 0
        else:
            return 0
    df['educ'] = df.apply(educ, axis=1)
    df.to_excel('BaseLimpiaEPH.xlsx', index=False)  # Guardamos la variable.
    print('Variable "educ" añadida.')

educ()

# Estadística descriptiva de la variable educ.
def est_descriptiva_educ():
    df = pd.read_excel('BaseLimpiaEPH.xlsx').fillna(0)
    df['educ'] = df['educ'].fillna(0)
    media = np.mean(df['educ']).round(2)
    var = np.var(df['educ'], ddof=1).round(2)
    desv = np.sqrt(var).round(2)
    min = np.min(df['educ'])
    p50 = np.percentile(df['educ'], 50)
    max = np.max(df['educ'])

    print(f'La media de años de educación es: {media}')
    print(f'La varianza de años de educación es: {var}')
    print(f'El desvío de años de educación es: {desv}')
    print(f'El mínimo de años de educación es: {min}')
    print(f'El máximo de años de educación es: {max}')
    print(f'El percentil 50 de años de educación es: {p50}')
    df.to_excel('BaseLimpiaEPH.xlsx', index=False)

est_descriptiva_educ()

# Para poder comparar el ingreso semanal de 2004 al de 2024, decidimos compararlos teniendo en base el aumento-
# del salario minimo vital y móvil:

# Fuente del salario minimo vital y movil 2004 Marzo: https://eco.mdp.edu.ar/cendocu/repositorio/00210-d.pdf
# Fuente del salario minimo vital y movil 2024 Marzo: https://www.boletinoficial.gob.ar/detalleAviso/primera/302927/20240222

# Salario mínimo vital y movil 2004 Marzo (en pesos): $350
# Salario mínimo vital y movil 2004 Marzo (en pesos): $202.800


def salario_semanal():
    SMVM2004 = 350
    SMVM2024 = 202800
    pp = round(SMVM2024 / SMVM2004, 2)
    print(f'El SMVM de 2024T1 es {pp} veces el SMVM de 2004T1')

    df = pd.read_excel('BaseLimpiaEPH.xlsx')
    df.loc[(df['ANO4'] == 2004) & (df['ESTADO'] == 1) & (df['P21'] > 0), 'salario_semanal'] = ((
        df.loc[(df['ANO4'] == 2004) & (df['ESTADO'] == 1) & (df['P21'] > 0), 'P21'] * pp) / 21.65) * 5
    df.loc[(df['ANO4'] == 2024) & (df['ESTADO'] == 1) & (df['P21'] > 0), 'salario_semanal'] = ((
        df.loc[(df['ANO4'] == 2024) & (df['ESTADO'] == 1) & (df['P21'] > 0), 'P21']) / 21.65) * 5
    df.to_excel('BaseLimpiaEPH.xlsx', index=False)
    print('Ajuste hecho')

    # Chequeamos los promedios para verificar que las cantidades no sean absurdas tras el ajuste.

    # Variables para el Histograma.
    sal_sem04 = df.loc[df['ANO4'] == 2004, 'salario_semanal'].dropna()
    sal_sem24 = df.loc[df['ANO4'] == 2024, 'salario_semanal'].dropna()
    df2004 = pd.DataFrame({'salario_semanal': sal_sem04, 'ANO4': '2004'})
    df2024 = pd.DataFrame({'salario_semanal': sal_sem24, 'ANO4': '2024'})
    combinado = pd.concat([df2004, df2024], ignore_index=True)
    combinado_filtrado = combinado[combinado['salario_semanal']
                                   < combinado['salario_semanal'].quantile(0.99)]
    prom04 = combinado_filtrado.loc[combinado_filtrado['ANO4']
                                    == '2004', 'salario_semanal'].mean()
    prom24 = combinado_filtrado.loc[combinado_filtrado['ANO4']
                                    == '2024', 'salario_semanal'].mean()
    print(f"Promedio semanal 2004 ajustado: {round(prom04, 2)}")
    print(f"Promedio semanal 2024: {round(prom24, 2)}")

    # Figura.
    fig, axes = plt.subplots(1, 2, figsize=(10, 6))

    # Histograma.
    sns.histplot(data=combinado_filtrado, x='salario_semanal', ax=axes[0], hue='ANO4', bins=25, multiple='stack', palette={
        '2004': '#5799c6', '2024': 'orange'})
    axes[0].set_title('Distribución de salarios semanales EPH 2004/2024')
    axes[0].set_xlabel('Salario Semanal (en pesos)')
    axes[0].set_ylabel('Frecuencia')
    axes[0].axvline(prom04, linestyle='dashed',
                    linewidth=1, color='red')
    axes[0].axvline(prom24, linestyle='dashed',
                    linewidth=1, color='green')
    axes[0].set_xticks(np.arange(0, 300001, 30000))
    # Se usa patches y line2D para personalizar los labels.
    parche_2004 = mpatches.Patch(color='#5799c6', label='2004')
    parche_2024 = mpatches.Patch(color='orange', label='2024')
    line_prom04 = Line2D([0], [0], color='red',
                         linestyle='--', label='Promedio 2004')
    line_prom24 = Line2D([0], [0], color='green',
                         linestyle='--', label='Promedio 2024')
    axes[0].legend(handles=[parche_2004, parche_2024, line_prom04,
                            line_prom24], title='Año y Promedios')

    # Kernels. Filtramos años para 2004/2024 y sus respectivos ingresos para la edad.
    panel_b_data = df[(df['ESTADO'] == 1) & df['salario_semanal'].notna() & df['CH06'].notna()]
    panel_b_data = panel_b_data[(panel_b_data['salario_semanal'] > 0) & (panel_b_data['salario_semanal'] < 300000)]
    
    sns.kdeplot(
        data=panel_b_data,
        x='CH06', 
        y='salario_semanal',  
        fill=True,
        cmap='magma_r',  
        bw_adjust=1,
        ax=axes[1]
    )
    
    axes[1].set_title('B) KDE Edad vs Salario Semanal')
    axes[1].set_xlabel('Edad')
    axes[1].set_ylabel('Salario Semanal (pesos)')
    axes[1].set_ylim(0, 300000)

    plt.tight_layout()
    plt.savefig('Histo_kernels salario', dpi=300)
    plt.show()


salario_semanal()

##Creamos la variable 'horastrab' y hacemos estadística descriptiva
variables_a_limpiar = ['PP3E_TOT', 'PP3F_TOT']

df = pd.read_excel('BaseLimpiaEPH.xlsx')
df[variables_a_limpiar] = df[variables_a_limpiar].replace([99, 999, 9999], np.nan)

# Variable 'horastrab'
df['horastrab'] = df['PP3E_TOT'] + df['PP3F_TOT']

estadisticas_horas = df['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
estadisticas_horas.index = ['Promedio', 'Desviación estándar', 'Mínimo', 'Mediana (p50)', 'Máximo']

print("Estadísticas descriptivas de horastrab:\n")
print(estadisticas_horas)

estad_2004 = df[df_eph['ANO4'] == 2004]['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
print("Estadísticas descriptivas - Año 2004:")
print(estad_2004)

estad_2024 = df[df['ANO4'] == 2024]['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
print("\nEstadísticas descriptivas - Año 2024:")
print(estad_2024)

df_pea_edad = df[(df['CH06'] >= 15) & (df['CH06'] <= 65) & (df['ESTADO'].isin([1, 2]))]

pea_edad_2004 = df_pea_edad[df_pea_edad['ANO4'] == 2004]
pea_edad_2024 = df_pea_edad[df_pea_edad['ANO4'] == 2024]

estad_pea_2004 = pea_edad_2004['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]
estad_pea_2024 = pea_edad_2024['horastrab'].describe(percentiles=[0.5])[['mean', 'std', 'min', '50%', 'max']]

print("Estadísticas descriptivas - PEA (15-65 años) 2004:\n", estad_pea_2004)
print("\nEstadísticas descriptivas - PEA (15-65 años) 2024:\n", estad_pea_2024)

##Resumen para la región NEA
df = pd.read_excel('BaseLimpiaEPH.xlsx')

# Filtrar la base para la región NEA
df_nea = df[df['REGION'] == 41]

# Función resumen por año ajustada
def resumen_por_año(df, año):
    df_año = df[df['ANO4'] == año]
    total_obs = df_año.shape[0]
    nas_estado = (df_año['ESTADO'] == 0).sum()
    ocupados = (df_año['ESTADO'] == 1).sum()
    desocupados = (df_año['ESTADO'] == 2).sum()
    variables_limpias = df_año.dropna(axis=1).shape[1]
    return total_obs, nas_estado, ocupados, desocupados, variables_limpias

res_2004 = resumen_por_año(df_nea, 2004)
res_2024 = resumen_por_año(df_nea, 2024)

# Totales
total_obs = df_nea.shape[0]
nas_estado_total = (df_nea['ESTADO'] == 0).sum()
ocupados_total = (df_nea['ESTADO'] == 1).sum()
desocupados_total = (df_nea['ESTADO'] == 2).sum()
variables_limpias_total = df_nea.dropna(axis=1).shape[1]

# Tabla resumen
tabla = pd.DataFrame({
    '2004': res_2004,
    '2024': res_2024,
    'Total': [total_obs, nas_estado_total, ocupados_total, desocupados_total, variables_limpias_total]
}, index=[
    'Cantidad observaciones',
    'Cantidad de observaciones con Nas en la variable "Estado"',
    'Cantidad de Ocupados',
    'Cantidad de Desocupados',
    'Cantidad de variables limpias y homogeneizadas'
])

print(tabla)


## Matriz de correlación
# Calculamos la matriz de correlaciones
correlacion = nea_seleccion.corr()

nombres_variables = {
    'CH06': 'Edad',
    'edad2': 'Edad²',
    'educ': 'Años de Educación',
    'salario_semanal': 'Salario Semanal',
    'horastrab': 'Horas Trabajadas'
}

correlacion.rename(columns=nombres_variables, index=nombres_variables, inplace=True)

# Grafico
plt.figure(figsize=(10, 7))
sns.heatmap(correlacion, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title("Matriz de Correlaciones - Región NEA")
plt.tight_layout()
plt.savefig("matriz_correlaciones_NEA.png", dpi=300)
plt.show()


##PCA
df = pd.read_excel('BaseLimpiaEPH.xlsx')
df['horastrab'] = df['PP3E_TOT'] + df['PP3F_TOT']

variables = ["CH06", "edad2", "educ", "salario_semanal", "horastrab"]
df_eph = df[variables].dropna() # Se agarran solo las variables y elimina valores faltantes

# Transformación de variables (media 0, desvío 1) 
scaler = StandardScaler(with_std=True, with_mean=True) 
eph_estandar = pd.DataFrame(scaler.fit_transform(df_eph), columns=df_eph.columns)

print("Promedio luego de la transformación:\n", eph_estandar.mean())
print("\nDesvío estándar luego de la transformación:\n", eph_estandar.std())



pca = PCA()
eph_pca = pca.fit_transform(eph_estandar)



scores = eph_pca
print(scores)

# GRAFICO DE DISPERSIÓN 

# Componentes a graficar
i, j = 0, 1  # PC1 y PC2

# Creamos el gráfico
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(scores[:,0], scores[:,1], alpha=0.5, s=5)
ax.set_xlabel("Componente Principal %d" % (i+1))
ax.set_ylabel("Componente Principal %d" % (j+1))
ax.set_title("Análisis de los componentes principales 1 y 2")
ax.set_ylim(-2, 6)  # Ajustar según el rango donde estén la mayoría de los puntos
ax.set_xlim(-2.5, 6)

# Escalamos los loadings para que sean visibles
scale_factor = 5

for k in range(pca.components_.shape[1]):
    x = pca.components_[i, k] * scale_factor
    y = pca.components_[j, k] * scale_factor
    ax.arrow(0, 0, x, y, color="red", head_width=0.2)
    ax.text(x * 1.1, y * 1.1, eph_estandar.columns[k], color="black", ha='center', va='center', fontweight='bold')



# % de la Varianza explicada por los componentes 
print(pca.explained_variance_ratio_)

# Análisis de la varianza explicada por los componentes

# Gráfico: varianza explicada y acumulada
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
ticks = np.arange(1, pca.n_components_ + 1)

for i, v in enumerate(pca.explained_variance_ratio_):
    axes[0].text(ticks[i], v + 0.02, f"{v:.2f}", ha='center')

for i, v in enumerate(np.cumsum(pca.explained_variance_ratio_)):
    axes[1].text(ticks[i], v + 0.02, f"{v:.2f}", ha='center')

# Subplot A: proporción individual de varianza explicada
axes[0].plot(ticks, pca.explained_variance_ratio_, marker="o", color="red")
axes[0].set_title("Proporción de varianza explicada por componente")
axes[0].set_xlabel("Componente principal")
axes[0].set_ylabel("Proporción")
axes[0].set_xticks(ticks)
axes[0].set_ylim([0,1])

# Subplot B: suma acumulada de varianza explicada
axes[1].plot(ticks, np.cumsum(pca.explained_variance_ratio_), marker="o", color="green")
axes[1].set_title("Varianza explicada acumulada")
axes[1].set_xlabel("Componente principal")
axes[1].set_ylabel("Suma acumulada")
axes[1].set_xticks(ticks)
axes[1].set_ylim([0, 1.1])
fig

df = pd.read_excel('BaseLimpiaEPH.xlsx')
nea = df[df['REGION'] == 41]

variables = ['CH06', 'edad2', 'educ', 'salario_semanal', 'horastrab']
nea_seleccion = nea[variables]
nea_seleccion = nea_seleccion.dropna()



##Cluster
df = pd.read_excel('BaseLimpiaEPH.xlsx')

#Usamos los predictores Edad y Educación
df_cluster = df[['CH06', 'educ']].dropna()

# Normalizar los datos para mejorar el rendimiento de k-means
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Función para aplicar k-means y graficar
def graficar_kmeans(k, datos, original_data):
    kmeans = KMeans(n_clusters=k, n_init=20, random_state=42)
    clusters = kmeans.fit_predict(datos)

    plt.figure(figsize=(6, 5))
    sns.scatterplot(
        x=original_data['CH06'],
        y=original_data['educ'],
        hue=clusters,
        palette='Set1',
        legend='full'
    )
    plt.title(f'Clustering k-medias con k = {k}')
    plt.xlabel('Edad')
    plt.ylabel('Años de Educación')
    plt.legend(title='Cluster')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'cluster_k{k}.png', dpi=300)
    plt.show()

# Ejecutar para k = 2, 4 y 10
for k in [2, 4, 10]:
    graficar_kmeans(k, df_scaled, df_cluster)

#Cluster k=2 analizando edad, educación y estado ocupacional
df_cluster_k2 = df[['CH06', 'educ', 'ESTADO']].dropna()

# Normaliza datos
scaler = StandardScaler()
df_scaled_k2 = scaler.fit_transform(df_cluster_k2[['CH06', 'educ']])

# Ejecuta k-medias con k=2
kmeans_k2 = KMeans(n_clusters=2, n_init=20, random_state=42)
clusters_k2 = kmeans_k2.fit_predict(df_scaled_k2)

# Agregar cluster y estado al dataframe
df_cluster_k2['Cluster'] = clusters_k2

# Filtrar solo Ocupado (1) y Desocupado (2)
df_cluster_k2 = df_cluster_k2[df_cluster_k2['ESTADO'].isin([1, 2])]

# Diccionario de colores para el estado ocupacional
palette_estado = {
    1: 'green',    # Ocupado
    2: 'orange'   # Desocupado
}

# Diccionario para etiquetas de la leyenda
estado_labels = {
    1: 'Ocupado',
    2: 'Desocupado'
}

plt.figure(figsize=(8, 6))
scatter = sns.scatterplot(
    x='CH06',
    y='educ',
    hue='ESTADO',
    style='Cluster',
    markers=['o', 'X'],
    data=df_cluster_k2,
    palette=palette_estado
)

plt.title('Distribución de Edad y Años de Educación (k=2) con Estado Ocupacional')
plt.xlabel('Edad')
plt.ylabel('Años de Educación')

handles, labels = scatter.get_legend_handles_labels()
n_clusters = df_cluster_k2['Cluster'].nunique()
estado_handles = handles[1:n_clusters+1]
estado_labels_leg = [estado_labels[int(l)] for l in labels[1:n_clusters+1]]

plt.legend(handles=estado_handles, labels=estado_labels_leg, title='Estado de Ocupación')
plt.grid(True)
plt.tight_layout()
plt.savefig('grafico_k2_ocupacion.png', dpi=300)
plt.show()

#Dendrograma
variables_cluster = df[['CH06', 'edad2', 'educ', 'salario_semanal', 'horastrab']].dropna()
scaler = StandardScaler()
variables_normalizadas = scaler.fit_transform(variables_cluster)

# Clustering jerárquico
linked = linkage(variables_normalizadas, method='ward') 

plt.figure(figsize=(12, 6))
dendrogram(
    linked,
    truncate_mode='lastp',     # Muestra solo los últimos clusters grandes
    p=30,                      # Número de ramas a mostrar
    leaf_rotation=90.,
    leaf_font_size=10.,
    show_contracted=True,
    color_threshold=8,         
    above_threshold_color='gray'
)
plt.axhline(y=8, c='red', linestyle='--', label='Corte sugerido')  # Línea de corte visual
plt.title('Dendrograma - Clustering Jerárquico (últimos 30 clusters)')
plt.xlabel('Observaciones agrupadas')
plt.ylabel('Disimilitud')
plt.legend()
plt.tight_layout()
plt.savefig('dendrograma.png', dpi=300)
plt.show()